<a href="https://colab.research.google.com/github/finardi/tutos/blob/master/FaQuAD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q transformers datasets

In [245]:
import numpy as np
import pandas as pd
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig

import random 
rnd = random.Random()

In [182]:
path_data = "/content/drive/MyDrive/LLMs/FaQuAD/data/"
data = load_dataset("json", data_files=path_data+"dev.json")
data

  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['data', 'version'],
        num_rows: 1
    })
})

In [189]:
passage, question, answer = [],[],[]
for ix, document in enumerate(data['train']['data'][0]):
    for doc in document['paragraphs']:
        questions_and_answers = []
        for q_a in doc['qas']:
            passage.append(doc['context'])
            question.append(q_a['question'])
            answer.append(q_a['answers'][0]['text'])

df = pd.DataFrame({'context':passage, 'question':question, 'answer':answer})
df            

,context,question,answer
0,caberá ao coordenador de curso verificar se o ...,o que o acadêmico deve preencher?,requisitos
1,caberá ao coordenador de curso verificar se o ...,onde deve constar o conteúdo a ser avaliado?,plano de ensino
2,caberá ao coordenador de curso verificar se o ...,por quantos docentes deve ser composta a banca...,três
3,caberá à banca examinadora atribuir ao acadêmi...,qual a nota mínima para o acadêmico que requis...,nove
4,caberá à banca examinadora atribuir ao acadêmi...,qual o limite de dias para o acadêmico que req...,dez
...,...,...,...
58,o vínculo acadêmico com a ufms dar-se-á median...,como faço para não perder o vínculo acadêmico?,mediante renovação de matrícula ou trancamento...
59,"a disciplina é identificada por seu nome, códi...",qual a carga horária mínima do encontro presen...,10 horas
60,"a resolução no 401, de 22 de novembro de 2013,...",de quando é a resolução no 401?,22 de novembro de 2013
61,"a resolução no 401, de 22 de novembro de 2013,...",o que a resolução no 401 estabelece?,normas para a renovação de matrícula dos curso...


In [244]:
#easy way: all questions are unique, then we can use their index
assert len(df) == df.question.nunique()

unique_questions = df.question.unique()

df[df.question==unique_questions[K]]

,context,question,answer
27,"as instituições de educação superior, por meio...",por meio de quem as instituições de ensino sup...,colegiados superiores


In [243]:
def dynamyc_fewshot_examples(num_fewshot, question_id, data):
    all_index = np.arange(len(data))
    possible_index = all_index[all_index != question_id]
    fewshot_list_index = rnd.sample(possible_index.tolist(), num_fewshot)
    return fewshot_list_index

def fewshot_context(index, num_fewshot, dataframe=df):
    fewshot_list_index = dynamyc_fewshot_examples(
        num_fewshot=num_fewshot, 
        question_id=index, 
        data=dataframe
    )
    

    labeled_examples = "Responda a questão baseada no Contexto." + "\n\n"
    for ix, sample in enumerate(fewshot_list_index, 1):
        data_fewshot = dataframe[dataframe.index==sample]
        labeled_examples += f"Exemplo {ix}:\nContexto: {data_fewshot['context'].to_list()[0]}\n"
        labeled_examples += f"Pergunta: {data_fewshot['question'].to_list()[0]}\nResposta: {data_fewshot['answer'].to_list()[0]}"
        labeled_examples += '\n###\n'
    labeled_examples += f'Contexto: '
    
    data_ask = dataframe[dataframe.index==index]
    example = f"{data_ask['context'].to_list()[0]}\n"
    example += f"Pergunta: {data_ask['question'].to_list()[0]}\nResposta: "
    
    return labeled_examples + example

three_shot, four_shot = [], []
for ix in range(len(df)):
    three_shot.append(fewshot_context(index=ix, num_fewshot=3, dataframe=df))
    four_shot.append(fewshot_context(index=ix, num_fewshot=4, dataframe=df))
print(four_shot[0])

Responda a questão baseada no Contexto.

Exemplo 1:
Contexto: caberá à banca examinadora atribuir ao acadêmico uma nota final de zero vírgula zero (0,0) a dez vírgula zero (10,0), resultante da média aritmética das notas emitidas pelos membros da banca; e considerar aprovado aquele acadêmico que obtiver a nota mínima final nove (9,0). caberá ao coordenador de curso informar ao acadêmico, em até dez (10) dias úteis antes da data prevista para avaliação, o conteúdo e as formas de avaliação com os respectivos pesos, além de data(s), horário(s) e local(is) de realização do(s) exame(s). se aprovado na avaliação, o acadêmico obterá dispensa de cursar a disciplina, sendo registrada no seu histórico escolar a nota conferida pela banca examinadora e a carga horária total da disciplina. o registro no siscad deverá ser feito pela coordenação do curso do acadêmico.
Pergunta: qual a nota mínima para o acadêmico que requisitou abreviação de curso ser aprovado?
Resposta: nove
###
Exemplo 2:
Contexto:

# Average tokens 

In [269]:
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")

def get_input_lenghts(dataset, tokenizer):
    num_words = [len(x.split()) for x in dataset]
    num_words = torch.tensor(num_words, dtype=torch.float)
    tokens_lengths = [len(tokenizer.encode(seq)) for seq in dataset]
    tokens_lengths = torch.tensor(tokens_lengths, dtype=torch.float)
    
    return int(torch.ceil(num_words.mean()).item()), tokens_lengths

# -------------------------------------------------------------------- #
shot3_num_words, shot3_lengths = get_input_lenghts(three_shot, tokenizer)
mean_3shot = torch.ceil(shot3_lengths.mean()).item()

shot4_num_words, shot4_lengths = get_input_lenghts(four_shot, tokenizer)
mean_4shot = torch.ceil(shot4_lengths.mean()).item()

# -------------------------------------------------------------------- #
print()
print('--'*12)
print()
print(f'3-shot mean {shot3_num_words} words.')
print(f'3-shot mean {int(mean_3shot)} tokens.')
print()
print('--'*12)
print()
print(f'4-shot mean {shot4_num_words} words.')
print(f'4-shot mean {int(mean_4shot)} tokens.')
print()
print('--'*12)

Token indices sequence length is longer than the specified maximum sequence length for this model (2459 > 2048). Running this sequence through the model will result in indexing errors



------------------------

3-shot mean 630 words.
3-shot mean 1522 tokens.

------------------------

4-shot mean 790 words.
4-shot mean 1920 tokens.

------------------------
